# 1) Simulation with fixed parameters

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from BSDE import * 
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

In [3]:
T = 1.
K = 100.
r = 0.02
R = 0.06
p = 8
M = 0.7 * np.eye(p) + 0.3
S_init = 100.
mu = 0.06
sigma = 0.2
Q = 0

In [4]:
m = 8
N = 1000
l = 1.
p = 8
n_picard = 3
n_neighbors = np.floor(0.1 * N)
M_run = 20

In [ ]:
a = np.zeros(M_run)
timing = np.zeros_like(a)
for i in range (M_run):
    start = time.time()
    test_hd = BsdeHD(T, K, M, mu, Q, sigma, S_init, r, R)
    a[i] = test_hd.get_price_derivative(R, r, N, m,
                                             option_type='put', option_payoff='average',
                                             oType='European', n_picard=n_picard, n_neighbors=n_neighbors, l=l)
    a[i] = round(a[i], 4)
    elapsed = time.time() - start
    timing[i] = round(elapsed, 3)
    print (a[i], timing[i])

In [8]:
a.mean()

2.5746450000000003

In [ ]:
import pandas as pd

In [ ]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [Q, mu, R, r, sigma, T, K, S_init]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [ ]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard', 'nearest neighbors', 'l']
svalues = [N, m, M_run, n_picard, n_neighbors, l]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

In [ ]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [ ]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [ ]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

In [ ]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_derivative.csv')

# 2) Impact of number of Nearest Neighbors

In [ ]:
price_nn = np.zeros(len(list(range(1, 100, 4))))
for i, n_neighbors_i in enumerate(list(range(1, N, 50))):
    start = time.time()
    test_hd = BsdeHD(T, K, M, mu, Q, sigma, S_init, r, R)
    price_nn[i] = test_hd.get_price_derivative(R, r, N, m,
                                             option_type='call', option_payoff='geometric',
                                             oType='European', n_picard=n_picard, n_neighbors=n_neighbors_i, l=l)
    elapsed = time.time() - start

In [ ]:
price_nn

# 3) Impact of parameter l 

In [ ]:
price_l = np.zeros(len(list(range(1, 100, 4))))
for i, l_i in enumerate(list(range(1, 100, 4))):
    test = BsdeHD(T, K, M, mu, Q, sigma, S_init, r, R)
    price_l[i] = test_hd.get_price_derivative(R, r, N, m,
                                             option_type='call', option_payoff='geometric',
                                             oType='European', n_picard=n_picard, n_neighbors=n_neighbors, l=l_i)

# 4) Plotting Z against Stocks Input

In [ ]:
T = 0.5
m_time_discretization = 12
plt.rcParams['figure.figsize'] = (7*m_time_discretization,7*2)
K = 100
S0 = 100
sigma = 0.2
r = 0.04
N_particles = 10000
n_neighbors = 999
mu = 0.06
R = 0.06
q = 0.
test = BSDE(S0, K, T, mu, sigma, q)
price_derivative = test.get_price_derivative(R, r, N_particles, m_time_discretization, l=1., 
                                             n_neighbors=n_neighbors,use_display=True)

print (price_derivative)

In [ ]:
1000 * np.log(1000) * 0.05